卡特兰数（Catalan number）是一种经典的组合数学问题，它在计算机科学、统计学和物理学等领域中有着广泛的应用。

它的其中一种定义为：有一个大小为$n \times n$方格图左下角为 (0, 0)， 右上角为 n, n，从左下角开始每次都只能向右或者向上走一单位，不走到对角线上方（但可以触碰）的情况下到达右上角有多少可能的路径？

递推公式为：
$$
c_0=1, c_{n+1}=\frac{2(2n+1)}{n+2}C_n
$$

现在，在STARK101 toturial 同样的有限域设置下，Prover 想给 Verifier 证明卡特兰数列第100项的数字为1555040254。请您对上述计算约束做算术化。

文字说明思路，以及给出最终 composition poly 在$h_{111}$上的取值 $p(h_{111})$作为 proof of work。（小域 ⟨g⟩size取 128，大域 ⟨h⟩size取 1024）


In [58]:
import time
from field import FieldElement
from polynomial import interpolate_poly, X, Polynomial,prod
from merkle import MerkleTree
from channel import Channel

In [59]:
# 首先构建数列
a = [FieldElement(1)]    
for i in range(128):
    a.append(a[i] *2*(2*i+1)/(i+2))

# a
a[100]

1555040254

In [60]:
# 构建群元素
g=FieldElement.generator() ** (3 * 2**30 / 128)
G = []
for i in range(128):
    G.append(g**i)

In [61]:
# 3. 构建多项式
f = interpolate_poly(G, a[:-1])

100%|██████████| 128/128 [00:00<00:00, 1360.91it/s]


In [62]:
# 4. 构建陪集
w = FieldElement.generator()
h = w ** ((3 * (2**30)) // 1024)
H = [h**i for i in range(1024)]
eval_domain = [w * x for x in H]
f_eval = [f(d) for d in eval_domain]
# 5. 构建Merkle树，构建commitment
channel = Channel()
f_merkle = MerkleTree(f_eval)
channel.send(f_merkle.root)

构建约束

这里要将原式进行变换
$$
c_0=1, c_{n+1}=\frac{2(2n+1)}{n+2}C_n
$$

因为这里含有n在外面 所以要想办法 消除，这里具体是如何消除的还没搞清楚但是可以转换为这个形式
$$
c_n=\sum_{i=0}^n C_i\cdot C_{n-i}$$

In [63]:
numer0 = f - 1
denom0 = X - 1
p0 = numer0 / denom0
numer1 = f - 1555040254
denom1 = X - g**100
p1 = numer1 / denom1



In [64]:
M_points = [(g**i, FieldElement(i)) for i in range(100)]
# 这里取了一个数对，就是构建了一个g元素到0-100 的映射关系, 这是最简单直接的方式
M = interpolate_poly([p[0] for p in M_points], [p[1] for p in M_points])


100%|██████████| 100/100 [00:00<00:00, 2203.61it/s]


In [65]:
numer2 = f(g * X) * (M + 2) - (2*(2*M+1) * f(X))
denom2 = prod([X - g**i for i in range(100)])
p2 = numer2 / denom2


In [66]:
def get_cp(channel, p0, p1, p2):
    alpha0 = channel.receive_random_field_element()
    alpha1 = channel.receive_random_field_element()
    alpha2 = channel.receive_random_field_element()
    return alpha0 * p0 + alpha1 * p1 + alpha2 * p2

def cp_eval(channel, eval_domain, CP):
    result = []
    for d in eval_domain:
        result.append(CP(d))
    return result



In [71]:
cp = get_cp(channel, p0, p1, p2)

cp_eval_result = cp_eval(channel, eval_domain, cp)

In [72]:
cp(h**888)

1509927980